In [1]:
import os
import sys

from pathlib import Path
from rdkit import Chem as rdChem
from tempfile import NamedTemporaryFile, gettempdir

sys.path.append("../")

In [2]:
from pymol_docking_plugin.Docking_Engine import Pymol_Docking
from pymol_docking_plugin.Protein_Minimization import minimize_complex

In [3]:
protein = Path("./LAC3.pdb")
crystal = Path("./Crystal.sdf")
ligand = Path("./Ligand.sdf")

In [4]:
docking = Pymol_Docking(protein, ligand)

docked_path, protein_path = docking.run_smina_docking("Dock", "XXX")
protein_complex = docking.run_complex_minimization(protein_path, docked_path)

Running protein preparation
Preparing protein with ProtoSS ...
Job 1722961f-a8bb-46b7-9a8b-bd3b2d90e2c8 completed with success


2025-03-27 12:36:37,874 - pymol_docking_plugin.Docking_Engine - INFO - Preparing ligands from SDF


Running docking


2025-03-27 12:36:59,800 - pymol_docking_plugin.Docking_Engine - INFO - Running pose buster
2025-03-27 12:36:59,914 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-03-27 12:36:59,914 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2025-03-27 12:37:00,148 - posebusters.posebusters - INFO - Using default configuration for mode redock.
[12:37:00] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[12:37:00] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[12:37:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[12:37:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[12:37:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[12:37:01] Warnin



Compliance rate: 96.89




2025-03-27 12:37:03,608 - openmmforcefields.generators.template_generators - INFO - Requested to generate parameters for residue <Residue 222 (UNK) of chain 1>
2025-03-27 12:37:03,616 - openmmforcefields.generators.template_generators - INFO - Generating a residue template for [H]c1c(c(c(c(c1[H])[H])[C@@]([H])(C([H])([H])c2c(c(c(c(c2[H])[H])[H])[H])[H])N([H])C(=O)C([H])([H])[C@@]3(C(=O)N(c4c(c(c(c(c4O3)[H])[H])[H])[H])[H])[H])[H])[H] using gaff-2.11


In [13]:
import datamol as dm

mol = dm.read_sdf(ligand)[0]

In [ ]:
minimizer = minimize_complex(protein, mol)

In [19]:
before = minimizer["PDB_BEFORE"]
after = minimizer["PDB_AFTER"]

In [24]:
import biotite.structure as struc
from biotite.structure.io import pdb

In [33]:
def write_temp_pdb(content: str) -> str:
    """Write PDB content to a temporary file and return the file path."""

    tmp_file = NamedTemporaryFile(delete=False, suffix=".pdb")
    with open(tmp_file.name, "w") as f:
        f.write(content)
    
    return tmp_file.name

temp_pdb_path_before = write_temp_pdb(before)
temp_pdb_path_after = write_temp_pdb(after)

struct_1 = pdb.PDBFile.read(temp_pdb_path_before).get_structure(model=1)
struct_2 = pdb.PDBFile.read(temp_pdb_path_after).get_structure(model=1)

In [34]:
multistate = struc.stack([struct_1, struct_2])

In [40]:
pdb_file = pdb.PDBFile()
pdb_file.set_structure(multistate)
pdb_file.write("DIO.pdb")

In [ ]:
minimizer